# Lesson 06: OpenCV

OpenCV is a library that contains algorithms and functions that are related to and perform image processing and computer vision tasks.

In [2]:
import cv2

# Opening Images

In [3]:
img = cv2.imread('../images/test-image.png')
print(img.shape)

(512, 768, 3)


In [ ]:
cv2.imshow('img', img)

# press 'q' to close image or else it will not be properly closed.
cv2.waitKey(0)
cv2.destroyAllWindows()

# Changing Color Space

In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [42]:
cv2.imwrite('../images/gray.jpg', gray)

True

In [ ]:
cv2.imshow('img', gray)

# press 'q' to close image or else it will not be properly closed.
cv2.waitKey(0)
cv2.destroyAllWindows()

# Resizing

In [ ]:
height, width = img.shape[:-1]
big_image = cv2.resize(img, (2*height, 2*width))
cv2.imshow('img', big_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Cropping

In [ ]:
img_crop = img[0:500, 300:500]
cv2.imshow('img', img_crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Thresholding

Thresholding only takes grayscale image.

The first argument is the grayscale image, the second argument is the threshold value which is used to classify the pixel values. The third argument is the maximum value which is assigned to pixel values exceeding the threshold. OpenCV provides different types of thresholding which is given by the fourth parameter of the function.

The method returns two outputs. The first is the threshold that was used and the second output is the thresholded image.

In [ ]:
thresholded = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

cv2.imshow('img', thresholded[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Filters

In [31]:
import numpy as np

kernel = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
])

# here second argument means the depth of the output image. 
# If you set this argument to -1 then the output image will have the same depth as input image.
filtered = cv2.filter2D(img, -1, kernel)

cv2.imshow('img', filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Gaussian Blur

Takes img, kernel size, standard deviation. If std is 0 then it is calculated from kernel size.

In [34]:
gaussian = cv2.GaussianBlur(img, (5, 5), 0)

cv2.imshow('img', gaussian)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Edge Detection

## Canny Edge Detector

First argument is grayscale image, then lower threshold, upper threshold and then kernel size which is by default 3.

The edge pixels above the upper threshold are considered in an edge map and edge pixels below the threshold are discarded. The pixel in between the thresholds are considered only if they are connected to pixels in upper threshold. Thus we get a clean edge map.

In [35]:
edges = cv2.Canny(gray, 100, 200, 3)

cv2.imshow('img', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Feature Detection

## SIFT

In [7]:
sift_img = cv2.imread('../images/test-image.png')

sift_obj = cv2.xfeatures2d.SIFT_create()

# you can also pass a mask in place of None if you want to find features in a specific region.
keypoints = sift_obj.detect(gray, None)

drawn = cv2.drawKeypoints(gray, keypoints, sift_img)

cv2.imshow('img', sift_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Feature Matching

In [8]:
img_rot = cv2.imread('../images/test-image-rot.png')
gray_rot = cv2.cvtColor(img_rot, cv2.COLOR_BGR2GRAY)

In [18]:
import random

sift = cv2.xfeatures2d.SIFT_create()

# detectAndCompute will find keypoints as well as compute the descriptors
# descriptors store information about keypoints that make them distinguishable
kp, desc = sift.detectAndCompute(gray, None)
kp_rot, desc_rot = sift.detectAndCompute(gray_rot, None)

# this is will match the descriptors and find the ones that are similar
bf = cv2.BFMatcher()
matches = bf.knnMatch(desc, desc_rot, k=2)

good = []
for m, n in matches:
    # applying the ratio test to filter out the bad matches due to noise
    # since we set k=2 in bf.knnMatch() we have two matches for each keypoint
    # the first one is the best and the second one of the second best
    # we are decreasing the value of the worst match and checking whether its
    # value becomes smaller then the best match. If it does then that match is discarded
    if m.distance < 0.4 * n.distance:
        good.append([m])

random.shuffle(good)
image_match = cv2.drawMatchesKnn(img, kp, img_rot, kp_rot, good[:10], flags=2, outImg=None)

cv2.imshow('img', image_match)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Opening Videos

In [21]:
cam = cv2.VideoCapture(0)

while (cam.isOpened()):
    ret, frame = cam.read()
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

# Converting to Grayscale

In [22]:
cam = cv2.VideoCapture(0)

while (cam.isOpened()):
    ret, frame = cam.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

# Saving Video

In [40]:
cam = cv2.VideoCapture(0)
ret, frame = cam.read()

h, w = frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# 1st argument: where to save video, 2nd compression format, 3rd fps, 4th size of video
video_writer = cv2.VideoWriter('../images/gray_vid.mp4', fourcc, 25.0, (w, h))

while (cam.isOpened()):
    ret, frame = cam.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray = cv2.Canny(gray, 30, 35, 3)
    
    # have to convert back to BGR format because video needs 3 channels
    # this will not bring back the colors
    
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    video_writer.write(gray)
    
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
video_writer.release()
cv2.destroyAllWindows()

# Resources

- https://docs.opencv.org/master/index.html
- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html
- https://www.youtube.com/playlist?list=PLQVvvaa0QuDdttJXlLtAJxJetJcqmqlQq